In [18]:
import collections
import nbimporter
import basicSim
import userCF_01label
from tqdm import tqdm
import csv

In [20]:
def read_triples(file_path):
    """
    从指定文件中读取用户-物品-评分的三元组。
    
    参数：
        file_path (str): 指定文件路径，例如 'rating_index.tsv'。
        
    返回：
        list of tuples: 返回一个包含 (user, item, rating) 三元组的列表。
    """
    triples = []  # 初始化一个空列表用于存储三元组
    with open(file_path, 'r', newline='') as file:
        reader = csv.reader(file, delimiter='\t')  # 使用制表符分隔
        for row in reader:
            user, item, rating = map(int, row)  # 转换为整数类型
            triples.append((user, item, rating))  # 将三元组添加到列表中
    return triples

In [22]:
triples = read_triples("rating_index.tsv")
triples

[(195, 1273, 0),
 (185, 1, 0),
 (21, 735, 0),
 (243, 543, 0),
 (165, 717, 0),
 (297, 612, 1),
 (114, 941, 0),
 (252, 599, 1),
 (304, 571, 0),
 (5, 304, 0),
 (61, 5, 0),
 (285, 208, 1),
 (199, 644, 1),
 (209, 1174, 0),
 (223, 675, 0),
 (302, 980, 0),
 (121, 1468, 1),
 (193, 832, 0),
 (290, 1581, 1),
 (233, 808, 0),
 (118, 911, 1),
 (166, 652, 1),
 (298, 1213, 1),
 (290, 1158, 0),
 (307, 760, 1),
 (94, 1159, 0),
 (37, 877, 1),
 (101, 1507, 0),
 (62, 1392, 1),
 (159, 1517, 1),
 (49, 1029, 0),
 (300, 1483, 1),
 (224, 202, 1),
 (289, 117, 1),
 (96, 1300, 0),
 (156, 832, 1),
 (180, 276, 0),
 (277, 1137, 1),
 (275, 174, 0),
 (6, 865, 1),
 (9, 639, 1),
 (283, 1250, 1),
 (200, 1374, 0),
 (275, 251, 0),
 (286, 78, 1),
 (245, 1074, 1),
 (241, 679, 1),
 (248, 131, 1),
 (98, 150, 1),
 (177, 1341, 0),
 (250, 868, 1),
 (80, 732, 0),
 (259, 1310, 1),
 (24, 985, 1),
 (58, 1238, 1),
 (71, 838, 0),
 (86, 1489, 1),
 (289, 226, 1),
 (41, 1490, 1),
 (291, 967, 1),
 (114, 1316, 0),
 (19, 910, 0),
 (200, 234,

In [24]:
def getSet(triples):
    """
    根据用户-物品-评分三元组构建一个物品-用户集合字典。
    
    参数：
        triples (list of tuples): 每个元组包含 (user, item, rating)，表示用户对某个物品的喜欢与否。
        
    返回：
        dict: 一个字典，其中键是物品 ID，值是该物品被喜欢的用户集合。
    
    说明：
        1. 遍历输入的三元组列表 triples，每个三元组包含用户 ID、物品 ID 和喜欢与否。
        2. 对于每个物品，将用户 ID 添加到对应物品的受众用户集合中。
        3. 使用 collections.defaultdict 来初始化每个物品的用户集合，便于自动处理空集合。
    """
    item_users = collections.defaultdict(set)
    for u,i,r in triples:
        if r == 1:
            item_users[i].add(u)
    return item_users

In [26]:
item_users = getSet(triples)
item_users

defaultdict(set,
            {612: {5,
              6,
              9,
              12,
              13,
              17,
              24,
              40,
              43,
              57,
              58,
              59,
              61,
              72,
              75,
              76,
              84,
              89,
              91,
              93,
              94,
              95,
              113,
              117,
              137,
              143,
              147,
              150,
              153,
              159,
              187,
              188,
              193,
              197,
              214,
              231,
              233,
              234,
              236,
              238,
              249,
              266,
              267,
              268,
              271,
              275,
              278,
              292,
              296,
              297,
              298,
              300,
              3

In [28]:
def knn4set_itemCF(trainset, k, sim_method):
    """
    计算每个物品在给定训练集中基于相似度的 K 相似物品。

    参数：
        trainset (dict): 一个字典，键是物品 ID，值是该物品的用户集合。
        k (int): 每个物品的近邻数量。
        sim_method (function): 一个相似度计算方法，用于计算两个物品之间的相似度。该方法接受两个集合作为输入，返回一个相似度值。

    返回：
        dict: 一个字典，其中键是物品 ID，值是与该物品最相似的 K 个物品的 ID 列表。

    说明：
        1. 对于训练集中的每个物品 i1，遍历所有其他物品 i2 以计算相似度。
        2. 只对拥有共同用户的物品对（i1 和 i2）进行相似度计算。
        3. 忽略 i1 自身以及没有共同用户的物品。
        4. 计算相似度后，将每个 i1 的所有其他物品按相似度排序并选取前 K 个物品作为其近邻。
    """
    sims = collections.defaultdict(list)
    for i1 in tqdm(trainset):
        ulist = []
        for i2 in trainset:
            if i1 == i2 or len(trainset[i1] & trainset[i2]) == 0:
                continue
            sim = sim_method(trainset[i1], trainset[i2])
            ulist.append((sim, i2))
        # 选择 K 个最相似的物品
        sims[i1] = [i[1] for i in sorted(ulist, reverse=True)[:k]]
    return sims

In [30]:
item_sims = knn4set_itemCF(item_users,k=3,sim_method=basicSim.Jaccard)
item_sims

100%|█████████████████████████████████████████████████████████████████████████████| 1373/1373 [00:04<00:00, 302.91it/s]


defaultdict(list,
            {612: [1307, 201, 956],
             599: [79, 372, 556],
             208: [872, 571, 728],
             644: [349, 985, 1370],
             1468: [807, 304, 540],
             1581: [870, 1540, 26],
             911: [685, 1141, 1299],
             652: [1202, 1070, 284],
             1213: [278, 285, 310],
             760: [839, 985, 310],
             877: [1423, 450, 1019],
             1392: [934, 1407, 1556],
             1517: [1483, 1088, 159],
             1483: [310, 1056, 246],
             202: [185, 1490, 895],
             117: [501, 2, 1090],
             832: [1155, 349, 857],
             1137: [68, 273, 1307],
             865: [1350, 201, 1435],
             639: [306, 106, 1349],
             1250: [575, 1221, 147],
             78: [1, 717, 694],
             1074: [1154, 234, 231],
             679: [1313, 821, 20],
             131: [272, 312, 1043],
             150: [2, 1502, 263],
             868: [839, 356, 1483],
            

In [38]:
def get_recommodations_by_itemCF(item_sims, user_o_set):
    """
    基于物品协同过滤（ItemCF）推荐物品。

    参数：
        item_sims (dict): 物品相似度字典，其中键是物品 ID，值是与该物品相似的物品 ID 列表。
        user_o_set (dict): 每个用户已拥有物品的集合，键是用户 ID，值是该用户已拥有的物品集合。

    返回：
        dict: 用户推荐字典，其中键是用户 ID，值是推荐给该用户的物品集合。

    功能：
        该函数遍历每个用户及其已拥有的物品，为每个物品找到相似物品，并将用户尚未拥有的相似物品推荐给用户。
    """
    
    recommodations = collections.defaultdict(set)
    for u in user_o_set:
        for item in user_o_set[u]:
            recommodations[u] |= set(item_sims[item]) - user_o_set[u]
    return recommodations

In [40]:
recommodations = get_recommodations_by_itemCF(item_sims,userCF_01label.getSet(triples))
recommodations

defaultdict(set,
            {297: {1,
              2,
              13,
              33,
              100,
              132,
              138,
              155,
              168,
              169,
              185,
              201,
              206,
              211,
              230,
              246,
              270,
              271,
              273,
              278,
              313,
              316,
              359,
              391,
              442,
              501,
              537,
              543,
              556,
              557,
              605,
              644,
              666,
              685,
              736,
              744,
              754,
              801,
              857,
              868,
              877,
              893,
              911,
              951,
              956,
              972,
              987,
              989,
              1017,
              1030,
              1056,
            